In [ ]:
import cv2
import numpy as np
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import img_to_array
from numpy import expand_dims
from matplotlib import pyplot as plt

# Load the VGG16 model
model = VGG16()
model.summary()
kernels, biases = model.layers[1].get_weights()
listparam = model.layers[1].get_config()

# Load and preprocess the image
# image = cv2.imread('./Lab3/image/B.jpeg')
image = cv2.imread('./Lab3/image/A.jpg')
if image is None:
    raise ValueError("Failed to load the image.")

img = cv2.resize(image, (224, 224))  # Resize to VGG16 input size
img = img_to_array(img)
img = expand_dims(img, axis=0)
img = preprocess_input(img)

# Create a model that outputs the feature maps of the first convolutional layer
conv1_layer = model.layers[1]
model_conv1 = Model(inputs=model.inputs, outputs=conv1_layer.output)
model_conv1.summary()

# Extract feature maps
feature_maps = model_conv1.predict(img)

print('kernel = ', kernels[0][0], '\nbiases = ', biases)
print(listparam)

# # Display the original image
# plt.subplot(1, 2, 1)
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.title('Original Image')

# # Display the preprocessed image
# plt.subplot(1, 2, 2)
# plt.imshow(img[0])
# plt.title('Preprocessed Image')

# plt.show()

# Plot the feature maps
plt.figure(figsize=(12, 12))
for i in range(feature_maps.shape[3]):
    plt.subplot(8, 8, i + 1)  # Adjust the subplot layout as needed
    plt.imshow(feature_maps[0, :, :, i],cmap='gray' )  # Display a single feature map
    plt.axis('off')
    plt.title(f'IMG {i + 1}')
plt.tight_layout()
plt.show()


: 

In [ ]:
import cv2
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import img_to_array
from numpy import expand_dims
from matplotlib import pyplot as plt

# Load the VGG16 model (optional, you can load it if you need it)
# model = VGG16()
# model.summary()
# kernels, biases = model.layers[1].get_weights()
# listparam = model.layers[1].get_config()



# Load and preprocess the image
image = cv2.imread('./Lab3/image/B.jpeg')

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB color space



image = cv2.resize(image, (224, 224))  # Resize to VGG16 input size

rs_image = np.reshape(image,(1,image.shape[0],image.shape[1],image.shape[2]))

img_mean = np.array([123.68, 116.779, 103.93])  # Mean pixel values
img_mean1 = np.array([103.93, 116.779, 123.68])  # Mean pixel values

print (image)
# print (img_mean1)
# rs_image1 = image - img_mean1  # Subtract mean values
# # print (rs_image1)
print("Preprocessed Image Pixel Values:")
print (rs_image)

brg_image = rs_image - img_mean # Subtract mean values
print ("AFTER = ", rs_image)
rgb_image1 = rs_image - img_mean1  # Subtract mean values

plt.subplot(1, 3, 1)  # 1 row, 2 columns, subplot 1
plt.imshow(image)
plt.title('Original Image')

plt.subplot(1, 3, 2)  # 1 row, 2 columns, subplot 2
plt.imshow(brg_image[0,:,:,:])
plt.title('Preprocessed Image')

plt.subplot(1, 3, 3)  # 1 row, 2 columns, subplot 2
plt.imshow(rgb_image1[0,:,:,:])
plt.title('Preprocessed Image')

plt.show()

In [ ]:
import cv2
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import img_to_array
from numpy import expand_dims
from matplotlib import pyplot as plt
from scipy import signal

# Load the VGG16 model
model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Load and preprocess the image
# image = cv2.imread('./Lab3/image/B.jpeg')
image = cv2.imread('./Lab3/image/A.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))
img = img_to_array(image)
img = expand_dims(img, axis=0)
img = preprocess_input(img)

# Get the weights (kernels) and biases from the first convolutional layer
layer = model.layers[1]
kernels, biases = layer.get_weights()

# Perform convolution for each color channel and each kernel
img_result = np.zeros((1, 224, 224, len(biases)))


img_mean = np.array([123.68, 116.779, 103.93])  # Mean pixel values

img = img - img_mean # Subtract mean values

for i in range(len(biases)):  # Loop over each filter
    for channel in range(img.shape[-1]):  # Loop over each color channel
        img_result[:, :, :, i] += signal.convolve2d(
            img[0, :, :, channel], kernels[:, :, channel, i], mode='same', boundary='fill', fillvalue=0
        )

# Add the biases to the convolutional results
for i in range(len(biases)):
    img_result[:, :, :, i] += biases[i]

# Apply ReLU activation function
img_result[img_result < 0] = 0
# img_result[img_result < 8] = 20

# Display the result of convolution and ReLU activation (64 feature maps)
plt.figure(figsize=(12, 12))
for i in range(64):
    plt.subplot(8, 8, i + 1)
    plt.imshow(img_result[0, :, :, i], cmap='gray')
    plt.axis('off')
    plt.title(f'IMG : {i + 1}')

plt.tight_layout()
plt.show()
